# MYSQL Database connector

Before running this notebook make sure the MySQL Python Connector is installed 
TODO: Add an examples of how to install it
TODO: Mention the environment variables are needed in order to connect to the mysql TPCH docker imagen

## Imports

In [1]:
import pydough
import os

## Loading credencials and connecting to MySQL

In [2]:
mysql_username = os.getenv("MYSQL_USERNAME")
mysql_password = os.getenv("MYSQL_PASSWORD")
mysql_tpch_db = os.getenv("MYSQL_DB")
mysql_host = os.getenv("MYSQL_HOST")

pydough.active_session.load_metadata_graph("../../tests/test_metadata/sample_graphs.json", "TPCH")
pydough.active_session.connect_database("mysql", 
        user=mysql_username,
        password=mysql_password,
        database=mysql_tpch_db,
        host=mysql_host,
)

DatabaseContext(connection=<pydough.database_connectors.database_connector.DatabaseConnection object at 0x10c18e710>, dialect=<DatabaseDialect.MYSQL: 'mysql'>)

In [3]:
%load_ext pydough.jupyter_extensions

In [6]:
%%pydough
# TPCH Q1
output = ( customers.CALCULATE(
    name
).TOP_K(5, by=name)
)

pydough.to_df(output)

,name
0,Customer#000000001
1,Customer#000000002
2,Customer#000000003
3,Customer#000000004
4,Customer#000000005
